In [1]:
import pandas as pd

In [6]:
import pandas as pd
import numpy as np
import base64
import datetime

def calc_bet(df,fc,fa,pc,valor,dias):
    df['date']=pd.to_datetime(df['date'])
    off_set_date=datetime.date.today() + datetime.timedelta(days=dias)
    df=df[df['date']<=off_set_date]
    df_rodada = df[
        [ 'league','home_team', 'away_team', 'Away', 'odds_away', 'my_away', 'Draw', 'odds_draw', 'my_draw', 'Home',
          'odds_home', 'my_home' ] ]
    df_away = df_rodada[ ['league', 'home_team', 'away_team', 'Away', 'odds_away', 'my_away' ] ]
    df_away[ 'aposta' ] = 'Away'
    df_away.columns = [ 'league','home_team', 'away_team', 'proba', 'odds', 'probaxodds', 'aposta' ]
    df_home = df_rodada[ [ 'league','home_team', 'away_team', 'Home', 'odds_home', 'my_home' ] ]
    df_home[ 'aposta' ] = 'Home'
    df_home.columns = [ 'league','home_team', 'away_team', 'proba', 'odds', 'probaxodds', 'aposta' ]
    df_draw = df_rodada[ [ 'league','home_team', 'away_team', 'Draw', 'odds_draw', 'my_draw' ] ]
    df_draw[ 'aposta' ] = 'Draw'
    df_draw.columns = [ 'league','home_team', 'away_team', 'proba', 'odds', 'probaxodds', 'aposta' ]
    df_1 = pd.concat([ df_away, df_draw, df_home ])
    df_1 = df_1.reset_index(drop=True)
    def set_aposta(x):
        if x >= fc:
            return 'conservador'
        elif x >= fa:
            return 'arrojado'
        else:
            return np.nan

    df_1[ 'tipo' ] = df_1[ 'proba' ].apply(lambda x: set_aposta(x))
    df_1.dropna(inplace=True)
    sum_conservaodor = df_1.loc[ df_1[ 'tipo' ] == 'conservador', 'probaxodds' ].sum()
    sum_arrojado = df_1.loc[ df_1[ 'tipo' ] == 'arrojado', 'probaxodds' ].sum()

    def valor_apostado(row):
        if row[ 'tipo' ] == 'conservador':
            x = valor * pc * (row[ 'probaxodds' ] / sum_conservaodor)
        else:
            x = valor * (1 - pc) * (row[ 'probaxodds' ] / sum_arrojado)
        return x

    df_1[ 'valor' ] = df_1.apply(lambda row: valor_apostado(row), axis=1).round(2)

    df_1=df_1[['league','home_team','away_team','odds','proba','aposta','tipo','valor']].reset_index(drop=True)
    return df_1.sort_values(by='league')

In [8]:
df=pd.read_csv('preview.csv')
calc_bet(df,0.7,0.5,0.6,100,2)

c:\users\pedro\anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  if __name__ == '__main__':
c:\users\pedro\anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\users\pedro\anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation

,league,home_team,away_team,odds,proba,aposta,tipo,valor
45,A-League,Brisbane Roar,Adelaide United,2.37,0.984395,Home,conservador,2.34
44,A-League,Perth Glory FC,Central Coast Mariners,1.49,0.966631,Home,conservador,2.14
8,Allsvenskan,Sirius,Norrköping,3.62,0.762525,Draw,conservador,2.15
41,Allsvenskan,Elfsborg,Djurgården,3.04,0.848572,Home,conservador,2.21
40,Allsvenskan,Malmö FF,Kalmar,1.34,0.821103,Home,conservador,1.83
10,Allsvenskan,Östersunds FK,Häcken,3.66,0.913602,Draw,conservador,2.45
9,Allsvenskan,Falkenberg,Helsingborg,3.12,0.855214,Draw,conservador,2.24
15,Championship,Stoke City,Brentford,5.59,0.819582,Home,conservador,2.64
16,Championship,Swansea City,Bristol City,1.85,0.793821,Home,conservador,1.88
34,Eliteserien,Rosenborg,Sandefjord,1.27,0.589514,Home,arrojado,1.63
